In [96]:
## IMPORTS
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import recordlinkage

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [97]:
## load in h2a data
h2a = pd.read_excel("./my_data/h2a_2018.xlsx")

## load in investigations/violations data
url = "https://enfxfr.dol.gov/data_catalog/WHD/whd_whisard_20210415.csv.zip"
investigations = pd.read_csv(url, 
                      index_col=None,
                      dtype={7:'string'})

## convert the dates to datetime objects
investigations['findings_start_date'] = pd.to_datetime(investigations['findings_start_date'], errors='coerce')
investigations['findings_end_date'] = pd.to_datetime(investigations['findings_end_date'], errors = "coerce")

In [98]:
## this function will pull out the certification status from a given h2a application
def find_status(one):
    string_version = str(one)                    ## convert to string
    pattern = r'\-\s(.*)$'                       ## define regex pattern
    found = re.findall(pattern, string_version)  ## search for pattern and return what's found
    return found[0]

h2a["status"] = [find_status(one) for one in h2a.CASE_STATUS]   ## put the status in a new column

## filter to applications that have received certification or partial certification
approved_only = h2a.loc[((h2a.status == "CERTIFICATION") | (h2a.status == "PARTIAL CERTIFICATION")),:].copy()

In [94]:
## try an exact merge to see if there are any exact matches
## print out a version without duplicate entries (just for visualization for this part)
merge_attempt = pd.merge(approved_only, investigations, how = "inner", left_on = "EMPLOYER_NAME", right_on = "legal_name", indicator = "name_merge_status") 
without_dups = merge_attempt[["EMPLOYER_NAME", "legal_name","CASE_STATUS","cty_nm","EMPLOYER_CITY","EMPLOYER_STATE","st_cd","h2a_violtn_cnt"]].drop_duplicates()
without_dups

,EMPLOYER_NAME,legal_name,CASE_STATUS,cty_nm,EMPLOYER_CITY,EMPLOYER_STATE,st_cd,h2a_violtn_cnt
0,Sandy Webster,Sandy Webster,DETERMINATION ISSUED - CERTIFICATION,Cedar City,Cedar City,UT,UT,0
1,Warren Bailey,Warren Bailey,DETERMINATION ISSUED - CERTIFICATION,Fountain Green,Fountain Green,UT,UT,2
2,"Grand Louis Four, Inc.","Grand Louis Four, Inc.",DETERMINATION ISSUED - CERTIFICATION,Mamou,Mamou,LA,LA,0
3,Justin Jonas,Justin Jonas,DETERMINATION ISSUED - CERTIFICATION,Christoval,Christoval,TX,TX,0
4,"Olson's Greenhouse Gardens, Inc.","Olson's Greenhouse Gardens, Inc.",DETERMINATION ISSUED - CERTIFICATION,Salem,Salem,UT,UT,123
...,...,...,...,...,...,...,...,...
993,"KCK Farms, LLC","KCK Farms, LLC",DETERMINATION ISSUED - CERTIFICATION,Dayton,Dayton,OR,OR,1
994,"JFT Harvesting, Inc.","JFT Harvesting, Inc.",DETERMINATION ISSUED - CERTIFICATION,Lake Placid,Lake Placid,FL,FL,0
995,"Farmers Gin Company, LLC","Farmers Gin Company, LLC",DETERMINATION ISSUED - CERTIFICATION,Saint Matthews,St. Matthews,SC,SC,0
998,Gustavo Cisneros,Gustavo Cisneros,DETERMINATION ISSUED - CERTIFICATION,Arcadia,Arcadia,FL,FL,14


In [99]:
## this function will clean the EMPLOYER_NAME in approved_only (h2a apps) and legal_name in violations (WHD data)
def clean_names(one):
    string_version = str(one)               ## convert to string
    upper_only = string_version.upper()     ## convert to uppercase
    pattern = r"(LLC|CO|INC)\."             ## locate the LLC, CO, or INC that are followed by a period
    replacement = r'\1'                     ## replace the whole pattern with the LLC/CO/INC component 
    res = re.sub(pattern, replacement, upper_only)  ## compute and return the result
    return res

## make new "name" columns for the cleaned versions of the names
approved_only["name"] = [clean_names(one) for one in approved_only.EMPLOYER_NAME]
investigations["name"] = [clean_names(one) for one in investigations.legal_name]
investigations_cleaned = investigations.loc[investigations.name != "NAN",:].copy()      ## get rid of NAN names

In [103]:
investigations_cleaned[["name","h2a_violtn_cnt"]]
violations = investigations_cleaned.loc[investigations_cleaned.h2a_violtn_cnt > 0, :].copy()

,name,h2a_violtn_cnt
0,"RELIANT ENERGY RETAIL SERVICES, LLC",0
1,"HEALTHCARE SERVICES GROUP, INC",0
2,"CENTRAL AVENUE BAKERY, INC",0
3,CATHOLIC CHERITIES,0
5,DERRICK PLUMBING,0
...,...,...
313922,"EL TAPATIO MEXICAN RESTAURANT(LA FINCA, INC)",0
313923,WEST FLEET INC,0
313924,NESBITT FRUIT FARMS,1
313925,"MOTOS, INC",0


In [36]:
## STEP 1: INITIALIZE OUR RECORDLINKAGE INDEX AND BLOCK ON STATE
link_apps_investigations = recordlinkage.Index()                                 
link_apps_investigations.block(left_on = "EMPLOYER_STATE", right_on = "st_cd")     
candidate_links_state = link_apps_investigations.index(approved_only, violations)

## STEP 2: INITIALIZE OUR COMPARE CLASS
compare = recordlinkage.Compare()        
compare.string("name","name", method = "jarowinkler", threshold = 0.85) 

## STEP 3: COMPUTE BASED ON OUR PROVIDED COMPARISON
compare_vectors = compare.compute(candidate_links_state, approved_only, violations)
compare_vectors.columns = ["Name"]                           ## rename the column to appropriate variable
selected = compare_vectors[compare_vectors.Name == 1].copy() ## select the matches

## STEP 4: PULL OUT THE INDEX VALUES SO THAT WE CAN MATCH THE CORRECT ENTRIES TO THE ORIGINAL DATA
n = selected.shape[0]
index_approved_only_values = []
index_violations_cleaned_values = []
for i in range(n):
    index = selected.index[i]
    index_approved_only_values.append(index[0])
    index_violations_cleaned_values.append(index[1])
selected["index_approved_only"] = index_approved_only_values.copy()
selected["index_violations_cleaned"] = index_violations_cleaned_values.copy()

## STEP 5: MERGE THE MATCHES BACK WITH THE APPROVED_ONLY H2A DATA
approved_only["index_approved_only"] = approved_only.index 
approved_only_columns = ["status","JOB_START_DATE","JOB_END_DATE","EMPLOYER_STATE", "name","index_approved_only"] 
m1 = pd.merge(selected, approved_only[approved_only_columns], on = "index_approved_only", how = "inner")

## STEP 6: MERGE THE RESULT FROM STEP 5 WITH THE VIOLATIONS DATA
violations["index_violations_cleaned"] = violations.index
violations_cleaned_columns = ["st_cd", "name", "index_violations_cleaned", "h2a_violtn_cnt","findings_start_date","findings_end_date"]
m2 = pd.merge(m1, violations[violations_cleaned_columns], on = "index_violations_cleaned", how = "inner", suffixes = ["_approved_only", "_violations_cleaned"])

## STEP 7: PRINT OUT MEANINGFUL SUBSET (HERE WE'RE FINIDNG MATCHES WITH MORE THAN 50 VIOLATIONS 
#                                       FOUND AND WITHIN THE CORRECT TIME RANGE)
apps_with_violations = m2.loc[((m2.findings_start_date >= m2.JOB_START_DATE)),:].copy()
apps_50_plus_violatons = m2.loc[(m2.h2a_violtn_cnt > 50) & ((m2.findings_start_date >= m2.JOB_START_DATE)),:].copy()

## DISPLAY
apps_with_violations.head()
apps_50_plus_violatons.head()


<Index>

<Compare>

,Name,index_approved_only,index_violations_cleaned,status,JOB_START_DATE,JOB_END_DATE,EMPLOYER_STATE,name_approved_only,st_cd,name_violations_cleaned,case_violtn_cnt,findings_start_date,findings_end_date
35,1.0,1841,300645,CERTIFICATION,2017-12-07,2018-05-31,CA,CARNEROS CREEK WINERY INC,CA,CARNEROS CREEK WINERY INC,19,2017-12-07,2018-07-23
36,1.0,1862,279472,CERTIFICATION,2017-12-14,2018-09-05,CA,PACIFIC LIVESTOCK INC,CA,"PACIFICA PERSONNEL, INC",114,2019-04-10,2019-06-03
37,1.0,3486,279472,CERTIFICATION,2017-12-14,2018-07-16,CA,PACIFICA PERSONNEL,CA,"PACIFICA PERSONNEL, INC",114,2019-04-10,2019-06-03
38,1.0,3694,279472,CERTIFICATION,2017-12-27,2018-07-16,CA,"PACIFICA PERSONNEL, INC",CA,"PACIFICA PERSONNEL, INC",114,2019-04-10,2019-06-03
39,1.0,3890,279472,CERTIFICATION,2018-01-08,2018-07-23,CA,PACIFICA PERSONNEL,CA,"PACIFICA PERSONNEL, INC",114,2019-04-10,2019-06-03


,Name,index_approved_only,index_violations_cleaned,status,JOB_START_DATE,JOB_END_DATE,EMPLOYER_STATE,name_approved_only,st_cd,name_violations_cleaned,case_violtn_cnt,findings_start_date,findings_end_date
36,1.0,1862,279472,CERTIFICATION,2017-12-14,2018-09-05,CA,PACIFIC LIVESTOCK INC,CA,"PACIFICA PERSONNEL, INC",114,2019-04-10,2019-06-03
37,1.0,3486,279472,CERTIFICATION,2017-12-14,2018-07-16,CA,PACIFICA PERSONNEL,CA,"PACIFICA PERSONNEL, INC",114,2019-04-10,2019-06-03
38,1.0,3694,279472,CERTIFICATION,2017-12-27,2018-07-16,CA,"PACIFICA PERSONNEL, INC",CA,"PACIFICA PERSONNEL, INC",114,2019-04-10,2019-06-03
39,1.0,3890,279472,CERTIFICATION,2018-01-08,2018-07-23,CA,PACIFICA PERSONNEL,CA,"PACIFICA PERSONNEL, INC",114,2019-04-10,2019-06-03
40,1.0,5250,279472,CERTIFICATION,2017-12-29,2018-07-16,CA,PACIFICA PERSONNEL,CA,"PACIFICA PERSONNEL, INC",114,2019-04-10,2019-06-03


In [102]:
def fuzzyMatch(dbase1, dbase2, blockLeft, blockRight, matchVar1, matchVar2, distFunction, threshold, colsLeft, colsRight):
    link_jobs_debar = recordlinkage.Index() ## initialize our Index
    link_jobs_debar.block(left_on = blockLeft, right_on = blockRight)         ## block on the given block variable

    ## form our index with the two given databases
    candidate_links = link_jobs_debar.index(dbase1, dbase2) 

    compare = recordlinkage.Compare()       ## initialize our compare class
    if (len(matchVar1) != len(matchVar2)):  ## ensure matching num. of matching vars
        print("Need to pass in your matching variables in an array and you need to have the same number of matching variables. Please try again. ")
        return

    for i in range(len(matchVar1)):         ## for each matching pair, add to our comparator
        compare.string(matchVar1[i], matchVar2[i], method = distFunction, threshold = threshold)

    compare_vectors = compare.compute(candidate_links, dbase1, dbase2) ## compute
    compare_vectors 

    # rename columns
    temp_array = []
    for i in range(len(matchVar1)):
        colName = str(matchVar1[i])
        temp_array.append(colName)
    compare_vectors.columns = temp_array

    ## Find the correct selection 
    conditions = []
    for one in matchVar1:
        condition_string = "({one_input} == 1)".format(one_input = one)
        conditions.append(condition_string)
    if (len(conditions) > 1):
        comparison = "&".join(conditions)
    else:
        comparison = conditions[0]
    selected = compare_vectors.query(comparison).copy()

    # Extract index from selection
    n = selected.shape[0]
    index_dbase1_values = []
    index_dbase2_values = []
    for i in range(n):
        index = selected.index[i]
        index_dbase1_values.append(index[0])
        index_dbase2_values.append(index[1])
    selected["index_dbase1"] = index_dbase1_values.copy()
    selected["index_dbase2"] = index_dbase2_values.copy()

    # merge jobs with original columns
    ## this will throw an error if jobs is not the left
    dbase1["index_dbase1"] = dbase1.index
    dbase1_columns = colsLeft
    m1 = pd.merge(selected, dbase1[dbase1_columns], on = "index_dbase1", how = "inner")

    # merge debar with original columns
    dbase2["index_dbase2"] = dbase2.index
    dbase2_columns = colsRight
    m2 = pd.merge(m1, dbase2[dbase2_columns], on = "index_dbase2", how = "inner", suffixes = ["_left", "_right"])

    return m2
    
#################################################################################################
approved_only["city"] = [one.upper() for one in approved_only.EMPLOYER_CITY]
violations["city"] = [one.upper() for one in violations.cty_nm]

#################################################################################################
blockLeft = "EMPLOYER_STATE"
blockRight = "st_cd"
matchingVarsLeft = ["name","city"]
matchingVarsRight = ["name","city"]
colsLeft = ["status","JOB_START_DATE","JOB_END_DATE","EMPLOYER_STATE", "name","index_dbase1","city"]
colsRight = ["st_cd", "name", "h2a_violtn_cnt","findings_start_date","findings_end_date","index_dbase2","city"]

res = fuzzyMatch(approved_only, violations, blockLeft,blockRight,matchingVarsLeft,matchingVarsRight,"jarowinkler",0.85,colsLeft,colsRight)

apps_50_plus_violations = res.loc[(res.h2a_violtn_cnt > 50) & ((res.findings_start_date >= res.JOB_START_DATE)),:].copy()
apps_50_plus_violations


,name_x,city_x,index_dbase1,index_dbase2,status,JOB_START_DATE,JOB_END_DATE,EMPLOYER_STATE,name_y,city_y,st_cd,name,h2a_violtn_cnt,findings_start_date,findings_end_date,city
30,1.0,1.0,3486,279472,CERTIFICATION,2017-12-14,2018-07-16,CA,PACIFICA PERSONNEL,SANTA MARIA,CA,"PACIFICA PERSONNEL, INC",113,2019-04-10,2019-06-03,SANTA MARIA
31,1.0,1.0,3694,279472,CERTIFICATION,2017-12-27,2018-07-16,CA,"PACIFICA PERSONNEL, INC",SANTA MARIA,CA,"PACIFICA PERSONNEL, INC",113,2019-04-10,2019-06-03,SANTA MARIA
32,1.0,1.0,3890,279472,CERTIFICATION,2018-01-08,2018-07-23,CA,PACIFICA PERSONNEL,SANTA MARIA,CA,"PACIFICA PERSONNEL, INC",113,2019-04-10,2019-06-03,SANTA MARIA
33,1.0,1.0,5250,279472,CERTIFICATION,2017-12-29,2018-07-16,CA,PACIFICA PERSONNEL,SANTA MARIA,CA,"PACIFICA PERSONNEL, INC",113,2019-04-10,2019-06-03,SANTA MARIA
34,1.0,1.0,5848,279472,CERTIFICATION,2018-01-26,2018-08-18,CA,"PACIFICA PERSONNEL, INC",SANTA MARIA,CA,"PACIFICA PERSONNEL, INC",113,2019-04-10,2019-06-03,SANTA MARIA
35,1.0,1.0,6150,279472,CERTIFICATION,2018-01-26,2018-07-17,CA,"PACIFICA PERSONNEL, INC",SANTA MARIA,CA,"PACIFICA PERSONNEL, INC",113,2019-04-10,2019-06-03,SANTA MARIA
36,1.0,1.0,6254,279472,CERTIFICATION,2018-02-02,2018-08-05,CA,"PACIFICA PERSONNEL, INC",SANTA MARIA,CA,"PACIFICA PERSONNEL, INC",113,2019-04-10,2019-06-03,SANTA MARIA
37,1.0,1.0,7349,279472,CERTIFICATION,2018-02-12,2018-08-23,CA,PACIFICA PERSONNEL,SANTA MARIA,CA,"PACIFICA PERSONNEL, INC",113,2019-04-10,2019-06-03,SANTA MARIA
38,1.0,1.0,9102,279472,CERTIFICATION,2018-03-07,2018-08-15,CA,"PACIFICA PERSONNEL, INC",SANTA MARIA,CA,"PACIFICA PERSONNEL, INC",113,2019-04-10,2019-06-03,SANTA MARIA
39,1.0,1.0,11656,279472,PARTIAL CERTIFICATION,2018-05-07,NaT,CA,"PACIFICA PERSONNEL, INC",SANTA MARIA,CA,"PACIFICA PERSONNEL, INC",113,2019-04-10,2019-06-03,SANTA MARIA
